# EndSARS Twitter Analysis

Sentiment and Network Analysis of the #EndSARS protest movement that occured in 2020 in Nigeria. 

## SetUp

### Import Python Libraries

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import yaml
import os
import json
import re
import ast
import csv
import gensim
from searchtweets import ResultStream, gen_request_parameters, load_credentials, collect_results
# import fastText as ft
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import datetime, timedelta


%matplotlib inline
pd.set_option("display.max_rows", 500)

### Connecting to Twitter

I suggest you use Twitter's official searchtweets api which is easy to use and reliable.
<br>In order to get past the tweets retrieval limit, you might also need to  use the premium search api.

- Create a hidden yaml file (.yaml) which contains details about your twitter api keys
- Create a variable that stores the twitter api credentials from the hiddem yaml file above.

**Keywords**: EndSARS, Soro Soke, Police Brutality, lekki toll gate, anti-robbery, lekki massacre, End bad governance,
End swat, buhari is a bad boy, sanwa-olu is a murderer, #LekkiMassacre

Helpful links:
- https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent
- https://github.com/twitterdev/search-tweets-python/tree/v2#using-the-twitter-search-apis-python-wrapper

In [82]:
bode_all_search_args = load_credentials(".twitter_keys.yaml",
                               yaml_key="search_tweets_all_bode_v2",
                               env_overwrite=False)

In [83]:
def retrieve_tweets(start_date, 
                    end_date,
                    max_tweets=30000,
                    results_per_call=100,
                    search_args=bode_all_search_args,
                    keywords_query="""#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR
"lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en""",
                    tweet_resp_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,\
                    entities,public_metrics", 
                    expansions_resp_fields="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,\
referenced_tweets.id,referenced_tweets.id.author_id", 
                    user_resp_fields="id,created_at,location,username,verified,public_metrics",
                    place_resp_fields="country,country_code,full_name,geo,id,name,place_type"):
    """
    Create a function that gets tweets using the premium search API where matching a certain keyword during a
    certain date range and returns all these tweets as a list
    """
    tweets_query = gen_request_parameters(query=keywords_query, 
                                          start_time=start_date, 
                                          end_time=end_date,
                                          expansions=expansions_resp_fields,
                                          tweet_fields=tweet_response_fields,
                                          user_fields=user_resp_fields,
                                          place_fields=place_resp_fields,
                                          results_per_call=results_per_call)
    tweets = collect_results(tweets_query,
                             max_tweets=max_tweets,
                             result_stream_args=search_args)
    return tweets

In [84]:
def get_dt_after(start_date, delta=timedelta(days=7)):
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = start_dt + delta
    return end_dt.strftime("%Y-%m-%d")

In [90]:
def create_tweets_df(date_list, 
                     max_tweets=30000,
                     results_per_call=100,
                     search_args=bode_all_search_args,
                     keywords_query="""#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR
"lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en""",
                     tweet_resp_fields="id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,\
                    entities,public_metrics", 
                     expansions_resp_fields="author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,\
referenced_tweets.id,referenced_tweets.id.author_id", 
                     user_resp_fields="id,created_at,location,username,verified,public_metrics",
                     place_resp_fields="country,country_code,full_name,geo,id,name,place_type"):
    """
    Create a function that Creates a dataframe from the tweets you've retrieved using the retieve_tweets function 
    you created above. This functions should also take a list of datest you want to get tweets from
    """
    tweets = []
    for date in dates_list:
        tweets += retrieve_tweets(start_date=date, end_date=get_dt_after(start_date=date),
                                  max_tweets=max_tweets, results_per_call=results_per_call, 
                                  search_args=search_args, keywords_query=keywords_query, 
                                  tweet_resp_fields=tweet_resp_fields, user_resp_fields=user_resp_fields, 
                                  place_resp_fields=place_resp_fields, expansions_resp_fields=expansions_resp_fields
                                 )
    df = pd.DataFrame(tweets)
    df.to_pickle(f"./data/endsars_tweets_{dates_list[0].replace('-', '')}_{dates_list[-1].replace('-', '')}.pkl")
    return df

In [86]:
get_dt_after("2021-02-22")

'2021-03-01'

In [91]:
tweets_df.to_pickle("./data/endsars_tweets_20201219_20201220.pkl")

## Fetch and Clean Data

Use the functions you've created above to collect all the tweets

In [92]:
dates_list = ["2020-12-01",  "2020-12-08"]

In [93]:
tweets_df = create_tweets_df(dates_list, max_tweets=10000)
tweets_df.shape

 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-01T00:00:00Z', 'end_time': '2020-12-08T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseufn23jwqe1f4gslo1fic7gy9h9'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"abou

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-01T00:00:00Z', 'end_time': '2020-12-08T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseufn1sw2i7w1wvzxvotkmeka8al'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-01T00:00:00Z', 'end_time': '2020-12-08T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseufn1i55qwag1qts6unnbmd23jx'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-01T00:00:00Z', 'end_time': '2020-12-08T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseufn1hz1ribkvuw9yevytfrinwd'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-01T00:00:00Z', 'end_time': '2020-12-08T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseufn17cpyb5gy6c17trdfku8xh9'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-08T00:00:00Z', 'end_time': '2020-12-15T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3fosevaazn04wx8zs3jugy2cjox9o59'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-08T00:00:00Z', 'end_time': '2020-12-15T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseva8xbarlkk7v6aeb6gcczfzxx9'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP Error code: 429: {"title":"Too Many Requests","type":"about:blank","status":429,"detail":"Too Many Requests"} | Too Many Requests
 Request payload: {'query': '#EndSARS OR "Soro Soke" OR "lekki toll gate" OR "anti-robbery squad" OR "lekki massacre" OR "End bad governance" OR "End swat" OR #LekkiMassacre lang:en', 'max_results': 100, 'start_time': '2020-12-08T00:00:00Z', 'end_time': '2020-12-15T00:00:00Z', 'tweet.fields': 'id,author_id,created_at,text,geo,referenced_tweets,in_reply_to_user_id,entities,public_metrics', 'user.fields': 'id,created_at,location,username,verified,public_metrics', 'place.fields': 'country,country_code,full_name,geo,id,name,place_type', 'expansions': 'author_id,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'next_token': 'b26v89c19zqg8o3foseva8wplczgrnjvyovrlc386bfjx'}
Rate limit hit... Will retry...
Will retry in 4 seconds...
 HTTP

Rate limit hit... Will retry...
Will retry in 4 seconds...


(20424, 17)

In [95]:
tweets_df.head()

,entities,created_at,author_id,public_metrics,text,id,referenced_tweets,in_reply_to_user_id,users,tweets,newest_id,oldest_id,result_count,next_token,geo,places,withheld
0,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",2020-12-07T23:59:59.000Z,1258938326591115271,"{'retweet_count': 8588, 'reply_count': 0, 'lik...",RT @Mistakobz: Second wave of #EndSARS protes...,1336098160754757632,"[{'type': 'retweeted', 'id': '1335887653993132...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",2020-12-07T23:59:59.000Z,921867476597649409,"{'retweet_count': 61, 'reply_count': 0, 'like_...",RT @Richchinonso: I’m officially resuming my j...,1336098160507281410,"[{'type': 'retweeted', 'id': '1335954601573556...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'mentions': [{'start': 3, 'end': 13, 'usernam...",2020-12-07T23:59:56.000Z,2507830334,"{'retweet_count': 890, 'reply_count': 0, 'like...","RT @EmekaGift: #WeMove, second wave of #EndSAR...",1336098147660021760,"[{'type': 'retweeted', 'id': '1335899145475940...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'mentions': [{'start': 3, 'end': 10, 'usernam...",2020-12-07T23:59:55.000Z,349431719,"{'retweet_count': 12664, 'reply_count': 0, 'li...",RT @Praiz8: Whenever you see any #EndSARS twee...,1336098144677982209,"[{'type': 'retweeted', 'id': '1335892081810812...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'mentions': [{'start': 3, 'end': 16, 'usernam...",2020-12-07T23:59:55.000Z,3303513131,"{'retweet_count': 993, 'reply_count': 0, 'like...",RT @Donteewrites: The Buhari administration fe...,1336098143138615296,"[{'type': 'retweeted', 'id': '1336028890247155...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Make sure to save the tweets after this process, use a file format that won't comprimise the characters in tweets

In [ ]:
def delete_duplicates(df, str_len=25):
    """
    Create a function that looks at the tweets in the tweets dataframe and other characters and delete 
    duplicates as sometimes we can get duplicate data/tweets. For example if I tweet something and someone else
    retweets it then this could possibly be a duplicate. Is this useful information? You need to decide of course.
    """

def preprocess_tweet(tweet):
    """
    Create a function that preprocess the text in a single tweet. Suggestions for this function are:
    - correct all multiple white spaces to a single white space
    - convert all urls to string "URL"
    - if the username isn't relevant then convert @username to "AT_USER"
    - converts emoji's to text description of the emoji
    """

In [ ]:
tweets_df = delete_duplicates(tweets_to_analyse)
tweets_df.reset_index(inplace=True, drop=True)
tweets_df['text_clean'] = tweets_df['text'].apply(preprocess_tweet)

## Alt data

In [ ]:
df = pd.read_excel('https://query.data.world/s/rghc6hq5b32ertwgebgwuyx2lcpxvt')

## Tweets Analysis

Now it's time to analyse this data to try and get some interesting insights. Below are some questions you can answer to get started but the best Data Scientists are creative thinkers so think out of th ebox to decide what will be interesting insight

**What locations in Nigeria and around the world were the hashtags popular?**

**How far did the hashtag reach and how deep was this reach?**

**When the did movement start on social media? When did it reach critical mass?**

**What were the most popular words or terms used during the movement?**

**Who were the key influences and proponents involved in the movement?**

**What was the most popular tweet?**

**What is the general sentiment of the tweets about the movement? How has this changed from the start of the movement till now?**

**Were there any bots or bad actors spreading fake news around the movement?**

**What are some examples of this fake news?**

## Sentiment Modelling
Train model on labelled twitter sentiment data from another dataset
<br>Next steps:
- Download Sentiment140 data
- preprocess tweet data
- Apply word embeddings to the text
- apply fasttext text classification
- Pickle model
- Apply model to endsars tweets text

### Get data ready for fastText training

Save as .txt file for the fastText classifer

In [ ]:
def prep_data(df, filename=""):
    """
    Create a functions that prepares the endsars data in a format that the fasttext model likes
    """

**Train Model**

**Apply Modelto Test data**

**Score Model**

**Check labels on example tweets**

### Apply trained fastText model to tweets df

In [ ]:
tweets_df['ft_predict'] = tweets_df.text.apply(ft_model.predict)
tweets_df["sentiment"] = tweets_df.ft_predict.apply(lambda x: 1 if x[0][0] == '__label__positive' else
                                                               0)
tweets_df['sentiment_prob'] = tweets_df.ft_predict.apply(lambda x: x[1][0])

In [ ]:
def print_tweet_sentiment(index=0):
    """
    Create function where you either pass the index of the tweet in the tweets_df or a tweet and it returns the 
    sentiment for the tweet
    """